# Abstract Factory

## O que é?

Esse padrão prevê a criação de uma interface única para para elementos que fazem parte de um grupo ou que dependem uns nos outros.

## Por quê?

Muitas vezes diferentes classes devem ser usadas em diferentes contextos ou em conjunto com diferentes objetos, ainda que elas desempenhem a mesma função. O padrão _abstract factory_ pertide classes específicas não sejam _hardcoded_, tornando o código mais flexível.

## Estrutura 

![estrutura](assets/estrutura.png)

## Exemplo 1

Uma companhia SaaS usa _machine learning_ para previnir fraude. Dois tipos de modelos são usados em produção, um simples model multiplicativo chamado _Logistic Regression_ e um modelo baseado em árvores de decisões chamado _Random Forest_. Cada modelo utiliza um método de optimização de parametros diferent, _Grid Search_ e _Random Search_ respectivamente.

Vamos ver como isso functionaria na prática.

In [6]:
import abc
import random

class BaseModel(abc.ABC):
    """
    Define the base interface for models
    """
    
    @abc.abstractmethod
    def fit(self, X: "List[List[float]]", y: "List[int]"):
        """
        Train the model
        """
        pass
    
    def predict(self, X):
        print(f"{self.__class__.__name__} has predicted something...")

class LogisticRegression(BaseModel):
    def __init__(self, l1_reg: float = 0, l2_reg: float = 1):
        self._l1_reg = l1_reg
        self._l2_reg = l2_reg
        
    def fit(self, X: "List[List[float]]", y: "List[int]"):
        print(f"Fitted using l1 of: {self._l1_reg} and l2 of {self._l2_reg}")
        
class RandomForest(BaseModel):
    def __init__(self, max_depth: int = 50, n_estimators: int = 20):
        self._max_depth = max_depth
        self._l2_reg = l2_reg
        
    def fit(self, X: "List[List[float]]", y: "List[int]"):
        print(
            f"Fitted using max_depth of: {self._max_depth} and {self._n_estimators} estimators"
        )
        
class Tuners(abc.ABC):
    """
    Defines the interface for hyper-parameter tuning classes.
    """
    @abc.abstractmethod
    def tune(self, params : dict) -> "BaseModel":
        """
        Tune the parameters of a model.
        """
        pass
        
class GridSearch:
    def __init__(self, model : "BaseModel"):
        self._model = model
              
    def tune(self, params : dict) -> "BaseModel":
        print("grid searching...")
        best_params = {params: random.random() for param in params}
        return model(**best_params)
        
class RandomSearch:
    def __init__(self, model : "BaseModel"):
        self._model = model

    def tune(self, params : dict) -> "BaseModel":
        print("randomly searching...")
        best_params = {params: random.randint() for param in params}
        return model(**best_params)

No dia a dia...

In [9]:
X = [list(range(3)) for _ in range(10)]
y = [random.randrange(0, 2) for _ in range(10)]

In [10]:
X

[[0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2]]

In [11]:
y

[0, 1, 1, 1, 1, 0, 0, 1, 0, 0]

In [ ]:
rf = RandomForest()

tuned_rf